<a href="https://colab.research.google.com/github/Sanjana-Sarma/NTTF_Training/blob/main/RL_Class_11thOct_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
pip install gym

In [87]:
import gym

env=gym.make("CartPole-v0")


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [88]:
num_episodes=100
num_timesteps=50

In [89]:
for i in range(num_episodes):
  Return=0
  state=env.reset()
  for t in range(num_timesteps):
    env.render()
    random_action=env.action_space.sample()
    next_state, reward, done, info = env.step(random_action)
    Return += reward
    if done:
      break

  if i%10==0:
    print(f"Episode: {i}, Return: {Return}")
env.close()


/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode: 0, Return: 15.0
Episode: 10, Return: 9.0
Episode: 20, Return: 39.0
Episode: 30, Return: 18.0
Episode: 40, Return: 21.0
Episode: 50, Return: 23.0
Episode: 60, Return: 16.0
Episode: 70, Return: 16.0
Episode: 80, Return: 42.0
Episode: 90, Return: 14.0


In [90]:
# List available environments
envs = gym.envs.registry.values()
env_ids = [env.id for env in envs]
print(env_ids)

['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4']


In [91]:
"""def epsilon_greedy_policy(state, epsilon):
  if random.uniform(0,1) < epsilon:
    return env.action_space.sample()
  else:
    return max(list(range(env.action_space.n)), key=lambda x:q[(space,x)])"""


'def epsilon_greedy_policy(state, epsilon):\n  if random.uniform(0,1) < epsilon:\n    return env.action_space.sample()\n  else:\n    return max(list(range(env.action_space.n)), key=lambda x:q[(space,x)])'

In [102]:
import gym
import pandas as pd
import random
from collections import defaultdict

In [103]:
env=gym.make("Blackjack-v1")
Q=defaultdict(float)
total_return=defaultdict(float)
N=defaultdict(int)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [104]:
def epsilon_greedy_policy(state, Q):
  epsilon=0.5
  if random.uniform(0,1) < epsilon:
    return env.action_space.sample()
  else:
    return max(list(range(env.action_space.n)), key=lambda x:Q[(state, x)])

In [105]:
num_timesteps=100
def generate_episodes(Q):
  episode=[]
  state=env.reset()
  for t in range(num_timesteps):
    action=epsilon_greedy_policy(state, Q)
    next_state, reward, done, info = env.step(action)
    episode.append((state, action, reward))
    if done:
      break
    state=next_state
  return episode

In [106]:
num_iterations=500000
for i in range(num_iterations):
  episode=generate_episodes(Q)
  all_state_action_pairs=[(s,a) for (s,a,r) in episode]
  rewards=[r for (s,a,r) in episode]
  for t, (state, action, _) in enumerate(episode):
    if not (state, action) in all_state_action_pairs[0:t]:
      R=sum(rewards[t:])
      total_return[(state,action)] += R
      N[(state,action)] += 1
      Q[(state,action)]=total_return[(state,action)]/N[(state,action)]

In [107]:
df=pd.DataFrame(Q.items(), columns=['state_action_pair', 'value'])

In [108]:
df.head(10)

,state_action_pair,value
0,"((21, 10, True), 0)",0.920244
1,"((20, 10, False), 0)",0.432413
2,"((20, 10, False), 1)",-0.890469
3,"((19, 9, False), 0)",0.266081
4,"((19, 9, False), 1)",-0.778043
5,"((16, 2, False), 0)",-0.268045
6,"((16, 2, False), 1)",-0.588565
7,"((7, 2, False), 0)",-0.252747
8,"((14, 10, False), 0)",-0.580772
9,"((13, 4, False), 0)",-0.250340


In [109]:
df[124:126]

,state_action_pair,value
124,"((14, 8, False), 1)",-0.494491
125,"((19, 6, False), 0)",0.525022
